In [1]:
abstract type Component end
abstract type Scatterer end

In [2]:
struct Structure <: AbstractVector{Component}
    components::Vector{Component}
end
Structure() = Structure([])

Base.size(s::Structure) = size(s.components)
Base.getindex(s::Structure, i::Int) = s.components[i]
Base.IndexStyle(::Type{<:Structure}) = IndexLinear()
function Base.push!(s::Structure, items...)
    for item in items
        push!(s.components, item)
    end
end
function Base.append!(s::Structure, arr::Array{N, 1} where N<:Component)
    for item in arr
        push!(s.components, item)
    end
end
Base.setindex!(s::Structure, c::Component, i::Int) = setindex!(s.components, c, i)

mutable struct SLD <: Scatterer
    real
    imag
    SLD(real) = new(real, 0.0)
    SLD(real, imag) = new(real, imag)
end

mutable struct Slab <: Component
    thickness
    scatterer::Scatterer
    roughness
    vfsolv
end

(|)(a::Component, b::Component) = Structure([a, b])
function |(s::Structure, b::Component)
    push!(s, b)
    s
end
function |(s::Structure, arr::Array{N, 1} where N<:Component)
    append!(s, arr)
    s
end

| (generic function with 3 methods)

In [3]:
s = Slab(1.0, SLD(2.0, 1), 3.0, 4.0)
t = Slab(1.0, SLD(2.0), 3.0, 4.0)
s, t

(Slab(1.0, SLD(2.0, 1), 3.0, 4.0), Slab(1.0, SLD(2.0, 0.0), 3.0, 4.0))

In [4]:
Structure([s, t])

2-element Structure:
 Slab(1.0, SLD(2.0, 1), 3.0, 4.0)
 Slab(1.0, SLD(2.0, 0.0), 3.0, 4.0)

In [7]:
p = s | t | s

3-element Structure:
 Slab(1.0, SLD(2.0, 1), 3.0, 4.0)
 Slab(1.0, SLD(2.0, 0.0), 3.0, 4.0)
 Slab(1.0, SLD(2.0, 1), 3.0, 4.0)

In [9]:
p |= 2

LoadError: MethodError: no method matching |(::Structure, ::Int64)
Closest candidates are:
  |(::Structure, !Matched::Component) at In[2]:37
  |(::Structure, !Matched::Array{N,1} where N<:Component) at In[2]:41

In [ ]:
println(objectid(p))
p = p | t
println(objectid(p))
p

In [ ]:
println(objectid(p))
p |= t
println(objectid(p))
p

In [ ]:
println(objectid(p))
p |= [s, t]
println(objectid(p))
p

In [ ]:
s==t, isequal(s, t), s===t

In [ ]:
s.thickness == t.thickness

In [ ]:
push!(p, s, t)
p

In [ ]:
append!(p, [s, t])
p

In [ ]:
for l in p
    println(l)
end

In [ ]:
eltype(p)

In [ ]:
t in p

In [ ]:
in(t, p)

In [ ]:
p |= [s, t]
p

In [ ]:
p |=  [s, t]
p

In [ ]:
typeof(p)

In [ ]:
objectid(p)

In [ ]:
p |= [s, t]
objectid(p)

In [ ]:
p | [s, t]

In [ ]:
objectid(p)

In [ ]:
items = [1,2,3]

In [ ]:
[item for item in items]

In [ ]:
[item for item in items]